# Data Wrangling for Scientists with Pandas

*From library documentation*: **pandas** is a fast, powerful, flexible and easy to use open source data analysis and manipulation tool,
built on top of the Python programming language.

In [ ]:
# SETUP: Import libraries and configure display
import pandas as pd
import numpy as np


# Configure pandas display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.precision', 2)


In [ ]:
# Input File path (origin: www.kaggle.com/)
input_file = "https://rcs.bu.edu/examples/python/DataAnalysis/Heart_Disease_Prediction.csv"

# Reading only the first 100 records for now
df = pd.read_csv(input_file)
df.head()


----

### Select Columns

In [ ]:
# Select single cell (returns a DataFrame)
clinical_age = df.loc[:, ['Age']]
# Alternatively, you can also use:
clinical_age = df.iloc[:, [0]]  # Assuming 'age' is the first column
# Or using direct column selection
clinical_age = df[['Age']]

print(f"\nFirst 5 rows:")
print(clinical_age.head())

In [ ]:
# Select multiple columns (returns a DataFrame)
clinical_params = df[['Age', 'BP', 'Cholesterol', 'Max HR']]
print(f"Type: {type(clinical_params)}")
print(f"\nFirst 5 rows:")
print(clinical_params.head())

----

### Filter rows

In [ ]:
# Filter rows based on a condition
# Example: Find patients with heart disease
heart_disease_patients = df[df['Heart Disease'] == 'Presence']
print(f"Patients with heart disease: {len(heart_disease_patients)}")
print(f"Percentage: {len(heart_disease_patients)/len(df)*100:.1f}%")

In [ ]:
# Multiple conditions with & (AND) and | (OR)
# Find elderly patients (age > 65) with high cholesterol (> 240)
elderly_high_chol = df[(df['Age'] > 65) & (df['Cholesterol'] > 240)]
print(f"Elderly patients with high cholesterol: {len(elderly_high_chol)}")
print(elderly_high_chol[['Age', 'Cholesterol', 'Heart Disease']].head())

In [ ]:
# Using .loc for label-based indexing
# Select specific rows and columns
subset = df.loc[0:4, ['Age', 'Sex', 'Cholesterol', 'Heart Disease']]
print("First 5 patients - selected columns:")
print(subset)

---

### Grouping and Aggregation

In biological research, we often need to compare groups (e.g., treatment vs control, disease vs healthy).

In [ ]:
# Group by heart disease status and calculate mean values
grouped_stats = df.groupby('Heart Disease')[['Age', 'Cholesterol', 'Max HR', 'BP']].mean()
print("Average clinical parameters by heart disease status:")
print(grouped_stats.round(2))

In [ ]:
# Multiple aggregations at once
agg_stats = df.groupby('Heart Disease')['Cholesterol'].agg(['mean', 'median', 'std', 'min', 'max'])
print("Cholesterol statistics by heart disease status:")
print(agg_stats.round(2))

In [ ]:
# Group by multiple columns
# Create sex labels for better readability
df['sex_label'] = df['Sex'].map({0: 'Female', 1: 'Male'})

multi_group = df.groupby(['Heart Disease', 'sex_label'])['Age'].agg(['count', 'mean'])
print("Age statistics by heart disease status and sex:")
print(multi_group.round(2))

---

### Creating New Columns and Data Transformation

Often we need to create new variables based on existing ones or transform data for analysis.

In [ ]:
# Create age groups
def categorize_age(age):
    if age < 40:
        return 'Young'
    elif age < 60:
        return 'Middle-aged'
    else:
        return 'Elderly'

df['age_group'] = df['Age'].apply(categorize_age)

# Check the distribution
print(df['age_group'].value_counts())
print("\nCross-tabulation:")
print(pd.crosstab(df['age_group'], df['Heart Disease']))

In [ ]:
df['risk_score'] = (
    (df['Age'] > 60).astype(int) * 2 +
    (df['Cholesterol'] > 240).astype(int) * 2 +
    (df['BP'] > 140).astype(int) * 1 +
    df['FBS over 120'] * 1 +
    df['Exercise angina'] * 2
)
df.head()

---

### Sorting and Ranking

Sorting data helps identify extremes and patterns.

In [ ]:
# Sort by cholesterol (descending)
high_chol = df.sort_values('Cholesterol', ascending=False).head(10)
print("Top 10 patients with highest cholesterol:")
print(high_chol[['Age', 'sex_label', 'Cholesterol', 'BP', 'Heart Disease']])

In [ ]:
# Sort by multiple columns
sorted_data = df.sort_values(['Heart Disease', 'risk_score'], ascending=[True, False])
print("Patients sorted by heart disease status and risk score:")
print(sorted_data[['Age', 'Sex', 'Heart Disease', 'risk_score']].head(10))

In [ ]:
df['risk_score'] = (
    (df['Age'] > 60).astype(int) * 2 +
    (df['Cholesterol'] > 240).astype(int) * 2 +
    (df['BP'] > 140).astype(int) * 1 +
    df['FBS over 120'] * 1 +
    df['Exercise angina'] * 2
)
df.head()

---

## Pivot Tables and Cross-Tabulation

Pivot tables help summarize data across multiple dimensions, similar to Excel pivot tables.

In [ ]:
# Create a pivot table
pivot = pd.pivot_table(df, 
                       values='Cholesterol', 
                       index='age_group', 
                       columns='Heart Disease', 
                       aggfunc='mean')
print("Average cholesterol by age group and heart disease status:")
print(pivot.round(2))

In [ ]:
# Crosstab with percentages
ct = pd.crosstab(df['age_group'], df['Heart Disease'], 
                 normalize='index') * 100
print("Percentage of heart disease by age group:")
print(ct.round(1))